### 📦 **Example 2 – Parametrized LP**

**Problem description:**
You have multiple products and resources:

* Each product has a profit.
* Each resource has a limit.
* Each product consumes resources at a certain rate.


In [ ]:
# Sets
"""
A simple linear programming example using Pyomo to maximize profit from product manufacturing.
"""
products = ['A', 'B', 'C']
resources = ['R1', 'R2']

# Parameters
"""
Product profits, resource limits, and consumption rates.
"""
profits = {'A': 40, 'B': 30, 'C': 50}
resource_limits = {'R1': 100, 'R2': 80}
consumption = {
    ('A','R1'): 2, ('A','R2'): 1,
    ('B','R1'): 1, ('B','R2'): 2,
    ('C','R1'): 3, ('C','R2'): 1
}

In [2]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory, NonNegativeReals, maximize, Set, Param

# Create the model
model = ConcreteModel()

# Sets
model.PRODUCTS = Set(initialize=products)
model.RESOURCES = Set(initialize=resources)

# Parameters
model.profit = Param(model.PRODUCTS, initialize=profits)
model.resource_limit = Param(model.RESOURCES, initialize=resource_limits)
model.consumption = Param(model.PRODUCTS, model.RESOURCES, initialize=consumption)

# Variables
model.production = Var(model.PRODUCTS, domain=NonNegativeReals)

# Objective
model.total_profit = Objective(expr = sum(model.profit[p] * model.production[p] for p in model.PRODUCTS), sense=maximize)

# Constraints
def resource_rule(model, r):
    return sum(model.consumption[p, r] * model.production[p] for p in model.PRODUCTS) <= model.resource_limit[r]
model.resource_constraint = Constraint(model.RESOURCES, rule=resource_rule)

# Solve
solver = SolverFactory('glpk')
result = solver.solve(model, tee=True)

# Print results
print("Optimal production quantities:")
for p in model.PRODUCTS:
    print(f"  {p}: {model.production[p].value}")
print("Total Profit:", model.total_profit())


GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\desai\AppData\Local\Temp\tmp97slojhb.glpk.raw --wglp C:\Users\desai\AppData\Local\Temp\tmpdpzajqmn.glpk.glp
 --cpxlp C:\Users\desai\AppData\Local\Temp\tmp1xax9a7o.pyomo.lp
Reading problem data from 'C:\Users\desai\AppData\Local\Temp\tmp1xax9a7o.pyomo.lp'...
2 rows, 3 columns, 6 non-zeros
27 lines were read
Writing problem data to 'C:\Users\desai\AppData\Local\Temp\tmpdpzajqmn.glpk.glp'...
19 lines were written
GLPK Simplex Optimizer, v4.65
2 rows, 3 columns, 6 non-zeros
Preprocessing...
2 rows, 3 columns, 6 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  3.000e+00  ratio =  3.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 2
*     0: obj =  -0.000000000e+00 inf =   0.000e+00 (3)
*     3: obj =   2.200000000e+03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (32380 bytes)
Writing basic 